<a href="https://colab.research.google.com/github/sud-hub/Cyclone-Intensity-Predictor/blob/main/Cyclone_Intensity_EfficientNet_B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sshubam_insat3d_infrared_raw_cyclone_images_20132021_path = kagglehub.dataset_download('sshubam/insat3d-infrared-raw-cyclone-images-20132021')

print('Data source import complete.')


100%|██████████| 43.2M/43.2M [00:00<00:00, 106MB/s] 

Extracting files...


Data source import complete.


In [ ]:
print(sshubam_insat3d_infrared_raw_cyclone_images_20132021_path)

/root/.cache/kagglehub/datasets/sshubam/insat3d-infrared-raw-cyclone-images-20132021/versions/1


In [ ]:
import os

# Option 1: Using os.listdir()
download_dir = sshubam_insat3d_infrared_raw_cyclone_images_20132021_path
print(os.listdir(download_dir))

# Option 2: Using shell command
!ls -F "$sshubam_insat3d_infrared_raw_cyclone_images_20132021_path"

['insat3d_ir_cyclone_ds', 'insat_3d_ds - Sheet.csv', 'insat3d_raw_cyclone_ds', 'insat3d_for_reference_ds']
'insat_3d_ds - Sheet.csv'    insat3d_ir_cyclone_ds/
 insat3d_for_reference_ds/   insat3d_raw_cyclone_ds/


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b0
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


In [ ]:
CSV_PATH = "/root/.cache/kagglehub/datasets/sshubam/insat3d-infrared-raw-cyclone-images-20132021/versions/1/insat_3d_ds - Sheet.csv"
BT_DIR = "/root/.cache/kagglehub/datasets/sshubam/insat3d-infrared-raw-cyclone-images-20132021/versions/1/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED"
RAW_DIR = "/root/.cache/kagglehub/datasets/sshubam/insat3d-infrared-raw-cyclone-images-20132021/versions/1/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL"

df = pd.read_csv(CSV_PATH)

# Ensure matching files in both folders
df = df[df['img_name'].apply(lambda x: os.path.isfile(os.path.join(BT_DIR, x)) and os.path.isfile(os.path.join(RAW_DIR, x)))]

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
class CycloneDataset(Dataset):
    def __init__(self, dataframe, bt_dir, raw_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.bt_dir = bt_dir
        self.raw_dir = raw_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filename = self.df.loc[idx, 'img_name']
        label = torch.tensor(self.df.loc[idx, 'label'], dtype=torch.float32)

        bt_img = Image.open(os.path.join(self.bt_dir, filename)).convert("RGB")
        raw_img = Image.open(os.path.join(self.raw_dir, filename)).convert("L")

        if self.transform:
            bt_img = self.transform(bt_img)
            raw_img = self.transform(raw_img)

        raw_img = raw_img[0].unsqueeze(0)  # (1, H, W)
        x = torch.cat([bt_img, raw_img], dim=0)  # (4, H, W)

        return x, label


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
])

train_ds = CycloneDataset(train_df, BT_DIR, RAW_DIR, transform)
val_ds = CycloneDataset(val_df, BT_DIR, RAW_DIR, transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)


In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights


def get_efficientnet_b0_4ch():
    weights = EfficientNet_B0_Weights.DEFAULT  # or use IMAGENET1K_V1 if you want exact match
    model = efficientnet_b0(weights=weights)

    # Modify first layer to accept 4 channels
    conv_stem = model.features[0][0]
    new_conv = nn.Conv2d(4, conv_stem.out_channels, kernel_size=3, stride=2, padding=1, bias=False)
    with torch.no_grad():
        new_conv.weight[:, :3, :, :] = conv_stem.weight  # Copy RGB weights
        new_conv.weight[:, 3:4, :, :] = conv_stem.weight[:, :1, :, :]  # Copy R to 4th channel
    model.features[0][0] = new_conv

    # Modify classifier for regression
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
    return model


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_efficientnet_b0_4ch().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

best_val_loss = float('inf')
patience = 10
wait = 0
EPOCHS = 50

for epoch in range(1, EPOCHS + 1):
    # Training
    model.train()
    train_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device).view(-1, 1)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    y_true, y_pred = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device).view(-1, 1)
            pred = model(x)
            loss = criterion(pred, y)
            val_loss += loss.item()

            y_true.extend(y.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"[Epoch {epoch:02d}] Train Loss: {avg_train_loss:.2f} | Val Loss: {avg_val_loss:.2f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")

    # Checkpointing
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        wait = 0
        torch.save(model.state_dict(), f"best_model_epoch{epoch}_mae{mae:.2f}.pt")
        print(f"✅ Saved best model at epoch {epoch}")
    else:
        wait += 1
        if wait >= patience:
            print(f"⏹️ Early stopping at epoch {epoch}")
            break


In [ ]:
model.load_state_dict(torch.load("/kaggle/working/best_model_epoch38_mae8.20.pt"))  # Replace with actual filename

model.eval()
preds, targets = [], []

with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device).view(-1, 1)
        outputs = model(x)
        preds.extend(outputs.cpu().numpy())
        targets.extend(y.cpu().numpy())

mae = mean_absolute_error(targets, preds)
rmse = np.sqrt(mean_squared_error(targets, preds))
print(f"📊 Final MAE: {mae:.2f}, RMSE: {rmse:.2f}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dummy predictions and targets for plotting (replace with actual values if needed)
# Here we assume `preds` and `targets` are already defined from the validation step

# Convert to flat NumPy arrays for plotting
preds_array = np.array(preds).flatten()
targets_array = np.array(targets).flatten()

# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(targets_array, preds_array, alpha=0.6, color='teal', edgecolors='k')
plt.plot([targets_array.min(), targets_array.max()], [targets_array.min(), targets_array.max()], 'r--', lw=2)
plt.xlabel("Actual Intensity (knots)")
plt.ylabel("Predicted Intensity (knots)")
plt.title("Predicted vs Actual Cyclone Intensity")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import torch

# Make sure model is in eval mode
model.eval()

# Get 16 samples from validation set
samples = [val_ds[i] for i in range(16)]
images = torch.stack([s[0] for s in samples])
labels = [s[1].item() for s in samples]

# Move to device
images = images.to(device)

# Predict
with torch.no_grad():
    preds = model(images).cpu().numpy().flatten()

# Plotting
plt.figure(figsize=(20, 20))
for i in range(16):
    img_tensor = samples[i][0][:3]  # Only take RGB channels
    img_np = img_tensor.permute(1, 2, 0).numpy()  # (C, H, W) → (H, W, C)
    img_np = img_np.clip(0, 1)  # Avoid overflow

    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(img_np)
    plt.title(f"O: {labels[i]:.1f}  |  P: {preds[i]:.1f}", fontsize=12)
    plt.axis("off")

plt.suptitle("Predicted vs Actual Cyclone Intensity (BT Image)", fontsize=20)
plt.tight_layout()
plt.show()
